In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
from isabelle_connector.isabelle_connector import IsabelleConnector

In [3]:
isabelle = IsabelleConnector(
    name="extract_transitions", working_directory=".", debug=True
)

### Missing Isabelle component: "/Users/yousef/.isabelle/contrib/hugo-0.119.0"


In [ ]:
from argparse import Namespace

from isabelle_connector.config import HOL_DIR, PROJ_ROOT

configs = Namespace(
    # dataset_name="yalhessi/lemexp",
    # dataset_config="hol-thms",
    root_dir=HOL_DIR,
    imports=[
        PROJ_ROOT / "isabelle-thys/ExtractLemmas",
        PROJ_ROOT / "template-eval/RoughSpec",
    ],
    # upload=True,
)

In [10]:
from isabelle_connector.extraction import transitions_theory
from isabelle_connector.isabelle_utils import get_theory, list_theory_files


thy_files = list_theory_files(configs.root_dir)
src_thys = [get_theory(theory_file, configs.root_dir) for theory_file in thy_files]
transition_thys = [transitions_theory(thy, configs) for thy in src_thys]


In [7]:
data, errs = isabelle.use_theories(transition_thys, batch_size=1000, recache=True)

Using cached results for 1462 / 1462 theories
Successful values from 1462 / 1462 theories
func:use_theories took: 8.909467935562134 sec


In [ ]:
# data["Transitions_ATP"]

[('ATP',
  [('<ignored>',
    '(*  Title:      HOL/ATP.thy\n    Author:     Fabian Immler, TU Muenchen\n    Author:     Jasmin Blanchette, TU Muenchen\n    Author:     Martin Desharnais, UniBw Muenchen\n*)\n\n'),
   ('section', 'section \\<open>Automatic Theorem Provers (ATPs)\\<close>'),
   ('<ignored>', '\n\n'),
   ('theory', 'theory ATP\n  imports Meson Hilbert_Choice\nbegin'),
   ('<ignored>', '\n\n'),
   ('subsection', 'subsection \\<open>ATP problems and proofs\\<close>'),
   ('<ignored>', '\n\n'),
   ('ML_file', 'ML_file \\<open>Tools/ATP/atp_util.ML\\<close>'),
   ('<ignored>', '\n'),
   ('ML_file', 'ML_file \\<open>Tools/ATP/atp_problem.ML\\<close>'),
   ('<ignored>', '\n'),
   ('ML_file', 'ML_file \\<open>Tools/ATP/atp_proof.ML\\<close>'),
   ('<ignored>', '\n'),
   ('ML_file', 'ML_file \\<open>Tools/ATP/atp_proof_redirect.ML\\<close>'),
   ('<ignored>', '\n\n\n'),
   ('subsection',
    'subsection \\<open>Higher-order reasoning helpers\\<close>'),
   ('<ignored>', '\n\n'),
 

In [ ]:
extraction_thys = [
    template_and_type_extraction_theory(thy, configs) for thy in src_thys
]

NameError: name 'template_extraction_theory' is not defined

In [ ]:
import ast

ast.literal_eval('"\<open>"')

<>:2: SyntaxWarning: invalid escape sequence '\<'
<>:2: SyntaxWarning: invalid escape sequence '\<'
/var/folders/sq/4j3r_9jx2qd6nxd1wdvz92200000gn/T/ipykernel_84754/3468013605.py:2: SyntaxWarning: invalid escape sequence '\<'
  ast.literal_eval('"\<open>"')
<unknown>:1: SyntaxWarning: invalid escape sequence '\<'


'\\<open>'